<a href="https://colab.research.google.com/github/JorgeSedek/tp2_orga_datos/blob/main/TP2_s4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 199 kB 46.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=96fc998570c23d03e04c1be3f217523490443f34dbd70fc77ef0971ecfe4a93c
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Ign:7 https://developer.download.nvid

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
rdd_movies = sqlContext.read.parquet('/content/drive/MyDrive/orgaDatos/Movie dataset/Parquets/movies.parquet').rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


S4. Para las películas que contienen el género de acción, si las separamos entre películas antes del 2000 y después del 2000, ¿cuál es el promedio de cantidad de votos para cada etapa?

In [ ]:
rdd_action = rdd_movies.filter(lambda x: not pd.isnull(x.genres) and ('Action' in x.genres) and (not pd.isnull(x.release_date)))

In [ ]:
rdd_selecc_2000 = rdd_action.flatMap(lambda x: [('antes2000' if (2000 < int(x.release_date.split('-')[0])) else 'igual2000' if (2000 == int(x.release_date.split('-')[0])) else 'despues2000', (x.vote_count, 1))])
lista_2000 = list(rdd_selecc_2000.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
.take(3))

In [ ]:
[(lista_2000[1][0],lista_2000[1][1][0] / lista_2000[1][1][1]) , (lista_2000[2][0],lista_2000[2][1][0] / lista_2000[2][1][1])]

[('antes2000', 420.26022304832713), ('despues2000', 99.51827956989247)]